# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем все необходимые библиотеки

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Сохраним **.csv** файл в датафрейм **df**

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


Датафрейм состоит из признаков и целевого признака, отсутствуют лишние (для обучения модели) столбцы

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Убедились что в данных нет пропусков и типы данных в каждом столбце числовы 

## Разбейте данные на выборки

Для начала разобьем данные на 60%(обучающая выборка) и 40% (валидационная и тестовая)

Затем поделим пополам валидационную и тестовую выборки

In [4]:
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=55555)

df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=55555)

Обучающая выборка содержит 1928 строк. Валидационная и тестовая по 643

In [5]:
print(df_train.shape, df_valid.shape, df_test.shape)

(1928, 5) (643, 5) (643, 5)


Поделим все наши выборки на признаки и целевые признаки.

Признаки - все столбцы кроме **'is_ultra'**

Целевой признак - столбец **'is_ultra'**

In [6]:
# обучающая выборка (признаки)
train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']

# валидационная выборка (признаки)
valid_features = df_valid.drop(['is_ultra'], axis=1)
valid_target = df_valid['is_ultra']

# тестовая выборка (признаки)
test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

## Исследуйте модели

Целевой признак у нас принимает значение 0 либо 1. Перед нами задача классификации(бинарная). 

Для предсказания будем использовать модели ***DecisionTreeClassifier*** и ***RandomForestClassifier***

In [7]:
# переменная для сохранения лучшей модели после подбора гиперпараметров для DecisionTreeClassifier
best_decision_tree_model = None 
# переменная для сохранения лучшего значения accuracy
decision_tree_accuracy = 0

# Цикл для перебора гиперпараметра max_depth(от 1 до 10) модели DecisionTreeClassifier
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=55555, max_depth=depth)
    model.fit(train_features, train_target)
    predict = model.predict(valid_features)
    accuracy = accuracy_score(valid_target, predict)
    if accuracy > decision_tree_accuracy:
            decision_tree_accuracy = accuracy
            best_decision_tree_model = model

# аналогичные переменные для модели RandomForestClassifier
best_random_forest_model = None
random_forest_accuracy = 0

# два вложенных цикла. Один для подбора гиперпараметра n_estimators, а второй для max_depth
for est in range(10, 51, 10):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=55555, max_depth=depth, n_estimators=est)
        model.fit(train_features, train_target)
        predict = model.predict(valid_features)
        accuracy = accuracy_score(valid_target, predict)
        if accuracy > random_forest_accuracy:
            random_forest_accuracy = accuracy
            best_random_forest_model = model

# вывод на экран лучшие модели с их accuracy
print(best_decision_tree_model, decision_tree_accuracy)
print(best_random_forest_model, random_forest_accuracy)

DecisionTreeClassifier(max_depth=4, random_state=55555) 0.7947122861586314
RandomForestClassifier(max_depth=8, n_estimators=20, random_state=55555) 0.8087091757387247


Получили две лучшие модели из DecisionTreeClassifier и RandomForestClassifier со значениями гиперпараметров. Обе модели на валидационной выборке дали результат выше 75%

С небольшим отрывом лучшей моделью становится RandomForestClassifier с гиперпараметрами: **n_estimators=20, max_depth=8**

## Проверьте модель на тестовой выборке

In [8]:
test = best_random_forest_model.predict(test_features)
accuracy_score(test_target, test)

0.8087091757387247

На тестовой выборке наша модель показывает результат 80.8% 

## (бонус) Проверьте модели на адекватность

Чтобы проверить на адекватность нужно сравнить ее результат со случайной моделью. Так как задача бинарной классификации, то можно, в качестве случайной модели, использовать генератор случайных чисел (0 и 1)

Для этого импортируем библиотеку **random**

In [9]:
import random

Случайно сгенерируем числа (0 и 1) в количестве строк целевого признака валидационной выборки(643) и сохраним в виде Series

Прокрутим такой генератор 100 раз для каждой выборки

In [10]:
valid_target.shape # количество строк целевого признака валидационной выборки

(643,)

In [11]:
summ = 0
count = 0

for i in range(100):  
    random_list = pd.Series([random.randint(0, 1) for _ in range(643)])
    summ += accuracy_score(valid_target, random_list)
    count += 1 
print(summ / count)

0.5004976671850699


после сотни генераций среднее accuracy случайных моделей всегда в районе 50%

In [12]:
test_target.shape # количество строк целевого признака тестовой выборки

(643,)

In [13]:
summ = 0
count = 0

for i in range(100):  
    random_list_1 = pd.Series([random.randint(0, 1) for _ in range(643)])
    summ += accuracy_score(test_target, random_list_1)
    count += 1 
print(summ / count)

0.4988802488335925


такой же результат и с тестовой выборкой 

# Вывод:

 В результате подбора моделей по гиперпараметрам лучшей оказалась модель RandomForestClassifier с гиперпараметрами: **n_estimators=20**, **max_depth=8**. Как на валидационной так и на тестовой выборке модель дала результат в **80.8 %.** Такой результат значительно выше случайно сгенерированных моделей которые показывают результат не выше 50%